In [1]:
import numpy as np
import torch

In [19]:
a = np.random.randint(1,10,size=(3,2,4,3))

In [25]:
b= np.random.randint(1,10,size=(2,3)).T

In [26]:
a

tensor([[[[8, 7, 4],
          [6, 7, 4],
          [9, 9, 2],
          [5, 3, 4]],

         [[2, 2, 7],
          [6, 9, 3],
          [5, 5, 2],
          [6, 2, 1]]],


        [[[8, 1, 6],
          [9, 8, 3],
          [3, 6, 7],
          [4, 9, 5]],

         [[4, 2, 2],
          [9, 3, 7],
          [3, 4, 5],
          [4, 1, 3]]],


        [[[5, 2, 9],
          [9, 6, 9],
          [6, 4, 1],
          [9, 8, 5]],

         [[1, 9, 6],
          [1, 8, 9],
          [6, 9, 2],
          [6, 2, 5]]]])

In [27]:
b

array([[6, 7],
       [2, 1],
       [6, 3]])

In [28]:
a,b = torch.from_numpy(a),torch.from_numpy(b)

TypeError: expected np.ndarray (got Tensor)

In [29]:
ans = torch.matmul(a,b)
ans

TypeError: matmul(): argument 'other' (position 2) must be Tensor, not numpy.ndarray

In [30]:
ans.shape

torch.Size([3, 2, 3, 2])

In [1]:
import numpy as np 
import open3d as o3d
import os
from natsort import natsorted
from glob import glob
from upsampling_mesh import meshify
from sklearn.neighbors import NearestNeighbors
import ckwrap 
import matplotlib.pyplot as plt
from scipy.io import savemat

o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

def find_neighs(points, k=2):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm= 'kd_tree').fit(points)
    distances, indices = nbrs.kneighbors(points) 
    return distances, indices

def fit_plane(points):
    # print(points.shape)
    dists_ori, indices_ori = find_neighs(points, k=250)
    pcd_neighs = points[indices_ori]
    # print(pcd_neighs.shape)
    norms_arr= []
    for pt in pcd_neighs:
        pca = PCA(n_components=3)
        pca.fit(pt)
        norms_arr.append(pca.components_[2])
    # print("norms_arr shape: ", np.array(norms_arr).shape)
    return norms_arr

def patching(points, normals, radius_chosen):
    dists_ori, indices_ori = find_neighs(points, k=250)
    dists_ori = dists_ori[:, 1:]
    indices_ori = indices_ori[:, 1:]
    
    # Apply ckmeans to all rows and extract labels using a lambda function:
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)

    # print("labels_arr shape: ",labels_per_row.shape)  # Now displays the actual labels for each row

    '''
    dists_array has 5 lists within, each list represents a cluster and has 2048 elements.
    '''
    dists_arrays = [[] for _ in np.unique(labels_per_row[0])]
    indices_arrays = [[]for _ in np.unique(labels_per_row[0])]
    for dist, ind, labels in zip(dists_ori, indices_ori, labels_per_row):
        # dist, ind, labels of shapes: (19,)
        for i in  np.unique(labels):
            temp_i = ind[np.where(labels == i)[0]]
            temp_d = dist[np.where(labels == i)[0]]
            dists_arrays[i].append(np.array([temp_d], dtype=object))
            indices_arrays[i].append(np.array([temp_i], dtype=object))
    return dists_arrays, indices_arrays


def visualize_clustered_points(dists_arrays):
    #visualising the clusters

    fig, ax = plt.subplots() 
    colors = ['blue', 'red', 'green', 'purple', 'orange']  
    for i, bucket in enumerate(dists_arrays):
        ax.plot(bucket[0], [i] * len(bucket[0]), '.', color=colors[i], markersize=10)

    plt.xlabel("Data Values")
    plt.ylabel("Cluster Labels")
    plt.title("Clustered Data")
    plt.grid(True)
    plt.show()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:

pcd_list = np.load('pcd_list.npy')
norm_list = np.load('normals_list.npy')
print("pcd_list shape: ", pcd_list.shape)

    

pcd_list shape:  (16867, 2048, 3)


In [10]:
pcd = pcd_list[10]